Importação de bibliotecas.

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta

Configurações de exibição do pandas

In [ ]:
pd.set_option('display.max_columns', None) # exibir todas as colunas
pd.set_option('display.width', 1000)      # aumentar a largura da exibição para evitar quebras de linha indesejadas

Carregando um arquivo CSV a partir de um caminho especificado.

In [ ]:
def carregar_dados(caminho_arquivo: str) -> pd.DataFrame:
    try:
        df = pd.read_csv(caminho_arquivo)
        if df.duplicated().any():
            print(f"aviso: encontradas e removidas {df.duplicated().sum()} linhas duplicadas em '{caminho_arquivo}'.")
            df = df.drop_duplicates(keep='first').reset_index(drop=True)
        return df
    except FileNotFoundError:
        print(f"erro: o arquivo '{caminho_arquivo}' não foi encontrado.")
        return pd.DataFrame()

Padronizando links de vídeo para um formato consistente. Ex: 'www.youtube.com' vira 'youtube.com', 'youtu.be' vira 'youtube.com/watch?v='.

In [ ]:
def normalizar_video_link(url: str) -> str:
    if pd.isna(url) or url == -1: # tratar valores NaN ou -1
        return url

    url_str = str(url).strip().lower() # limpa espaços e padroniza para minúsculas

    # normaliza links do twitch
    if 'twitch.tv/videos/' in url_str:
        return url_str.replace('www.', '').replace('m.', '')

    # normaliza links do youtube
    if 'youtube.com/watch?v=' in url_str:
        return url_str.replace('www.', '') # remove www.
    if 'youtu.be/' in url_str: # converte youtu.be para o formato watch?v=
        match = re.search(r'youtu\.be/([a-zA-Z0-9_-]+)', url_str)
        if match:
            video_id = match.group(1)
            return f'https://youtube.com/watch?v={video_id}'
        return url_str # retorna original se não conseguir extrair ID

    return url_str # retorna a url original se não for twitch/youtube ou já estiver normalizada

Processa dados do bilibili para calcular estatísticas de canal
5 dias antes e 5 dias depois de cada vídeo de recorde.



In [ ]:
# a ideia é carregar os dados do Bilibili e para cada vídeo de recorde
# calcular a média de visualizações e likes nos cinco dias anteriores e posteriores
def processar_recordes_bilibili_com_5d(caminho_csv: str) -> pd.DataFrame:
    print(f"\n--- Processando Bilibili a partir de: {caminho_csv} ---")

   #carregando o csv
    try:
        df = pd.read_csv(caminho_csv, parse_dates=["data_publicacao"])
        print(f"Bilibili: CSV carregado. {len(df)} linhas iniciais.")
    except FileNotFoundError:
        print(f"Bilibili ERRO: O arquivo '{caminho_csv}' não foi encontrado.")
        return pd.DataFrame()
    except Exception as e:
        print(f"Bilibili ERRO ao carregar CSV: {e}")
        return pd.DataFrame()

    # garantimos que a coluna de data esteja no formato certo
    df["data_publicacao"] = pd.to_datetime(df["data_publicacao"], errors="coerce")
    if df["data_publicacao"].isnull().any():
        print("Bilibili AVISO: Algumas 'data_publicacao' são inválidas e foram convertidas para NaT.")

    # filtrando para pegar apenas os vídeos marcados como recorde
    # o tratamento com strip e lower é para evitar problemas com espaços ou maiúsculas
    df_recordes = df[
        df["context_video"].astype(str).str.strip().str.lower() == "recorde"
    ].copy()
    print(f"Bilibili: {len(df_recordes)} vídeos com 'context_video' == 'recorde'.")

    # se não encontrarmos nenhum vídeo de recorde não há o que fazer
    if df_recordes.empty:
        print("Bilibili AVISO: Nenhum vídeo de 'recorde' encontrado após a filtragem. Retornando DataFrame vazio.")
        return pd.DataFrame()

    # aqui vamos guardar os resultados de cada vídeo de recorde processado
    resultados = []

    # passando por cada vídeo de recorde que encontramos
    for index, rec in df_recordes.iterrows():
        streamer = rec["name_streamer"]
        data_rec = rec["data_publicacao"]

        # verificação de segurança caso a data do recorde seja inválida
        if pd.isna(data_rec):
            print(f"Bilibili AVISO: Ignorando recorde do streamer '{streamer}' devido a 'data_publicacao' inválida.")
            continue

        # selecionando todos os vídeos do mesmo streamer na janela de tempo que nos interessa

        antes = df[
            (df["name_streamer"] == streamer)
            & (df["data_publicacao"] >= data_rec - timedelta(days=5))
            & (df["data_publicacao"] < data_rec)
        ]
        depois = df[
            (df["name_streamer"] == streamer)
            & (df["data_publicacao"] > data_rec)
            & (df["data_publicacao"] <= data_rec + timedelta(days=5))
        ]

        # calculando as médias para o período antes e depois
        # se não houver vídeos em um dos períodos o valor padrão será -1
        views_antes = int(antes["views"].mean()) if not antes.empty else -1
        likes_antes = int(antes["likes"].mean()) if not antes.empty else -1
        views_depois = int(depois["views"].mean()) if not depois.empty else -1
        likes_depois = int(depois["likes"].mean()) if not depois.empty else -1

        #montando um dicionário com os dados originais do recorde
        rec_final = rec.to_dict()
        # adicionando as novas colunas com as médias que calculamos
        rec_final.update({
            "Views_5d_Antes": views_antes,
            "Likes_5d_Antes": likes_antes,
            "Views_5d_Depois": views_depois,
            "Likes_5d_Depois": likes_depois
        })

        # adicionando o resultado completo à nossa lista de resultados
        resultados.append(rec_final)

    # transformando a lista de dicionários em um novo DataFrame
    df_final = pd.DataFrame(resultados)
    print(f"Bilibili: {len(df_final)} recordes processados e adicionados ao DataFrame.")
    return df_final

Carregando, padronizando e combinando os DataFrames de estatísticas de vídeo, garantindo que todas as colunas do YouTube existam em todos os DataFrames, preenchendo os valores ausentes com -1.






In [ ]:
def unificar_estatisticas_videos(path_twitch: str, path_youtube: str, path_bilibili: str) -> pd.DataFrame:

    df_twitch_raw = carregar_dados(path_twitch)
    df_youtube_raw = carregar_dados(path_youtube)

    processed_dfs = []

    # --- processar twitch ---
    if not df_twitch_raw.empty:
        df_twitch_temp = df_twitch_raw.rename(columns={
            'url': 'video_link', 'views': 'video_views', 'title': 'video_titulo'
        }).copy()
        df_twitch_temp['plataforma'] = 'Twitch'
        df_twitch_temp['video_link'] = df_twitch_temp['video_link'].apply(normalizar_video_link)
        processed_dfs.append(df_twitch_temp)

    # --- processar youtube ---
    if not df_youtube_raw.empty:
        df_youtube_temp = df_youtube_raw.rename(columns={
            'Views': 'video_views',
            'Likes': 'video_likes',
            'Comments': 'video_comentarios',
            'PublishedAt': 'video_data_publicacao',
            'title': 'video_titulo'
        }).copy()
        df_youtube_temp['plataforma'] = 'YouTube'
        df_youtube_temp['video_link'] = df_youtube_temp['video_link'].apply(normalizar_video_link)
        colunas_redundantes = ['date', 'player', 'time_seconds', 'time_formatted', 'run_link']
        df_youtube_temp = df_youtube_temp.drop(columns=[c for c in colunas_redundantes if c in df_youtube_temp.columns], errors='ignore')
        processed_dfs.append(df_youtube_temp)

    # --- processar bilibili ---
    df_bilibili_processed = processar_recordes_bilibili_com_5d(path_bilibili)

    if not df_bilibili_processed.empty:
        df_bilibili_temp = df_bilibili_processed.rename(columns={
            'views': 'video_views',
            'likes': 'video_likes',
            'comments': 'video_comentarios',
            'data_publicacao': 'video_data_publicacao',
            'title': 'video_titulo'
        }).copy()
        df_bilibili_temp['video_link'] = 'https://www.bilibili.com/video/' + df_bilibili_temp['bvid'].astype(str)

        df_bilibili_temp['video_link'] = df_bilibili_temp['video_link'].apply(normalizar_video_link)

        df_bilibili_temp['plataforma'] = 'Bilibili'
        processed_dfs.append(df_bilibili_temp)

    # --- unificação ---
    if not processed_dfs:
        print("Aviso: Nenhum DataFrame de plataforma foi carregado corretamente.")
        return pd.DataFrame()

    df_stats_unificado = pd.concat(processed_dfs, ignore_index=True)
    df_stats_unificado = df_stats_unificado.fillna(-1)

    return df_stats_unificado

Juntando os dados, removendo duplicatas pós-junção e convertendo os tipos de dados.

In [ ]:
def juntar_e_limpar_dados(path_runs: str, df_stats: pd.DataFrame) -> pd.DataFrame:

    df_runs = carregar_dados(path_runs)
    # normalizar links das runs antes do merge
    df_runs['video_link'] = df_runs['video_link'].apply(normalizar_video_link)
    df_runs = df_runs.drop_duplicates(subset=['video_link'], keep='first')

    df_runs = df_runs.rename(columns={
        'date': 'run_date', 'player': 'run_player', 'time_seconds': 'run_time_seconds',
        'time_formatted': 'run_time_formatted', 'run_link': 'run_url'
    })

    # left merge para manter todas as runs, preenchendo o que não encontrar com NaN
    df_final = pd.merge(df_runs, df_stats, on='video_link', how='left')

    # conversão de tipos de dados para datas
    # o errors='coerce' transforma datas inválidas em NaT, que serão preenchidos
    df_final['run_date'] = pd.to_datetime(df_final['run_date'], errors='coerce')
    df_final['video_data_publicacao'] = pd.to_datetime(df_final['video_data_publicacao'], errors='coerce')

    # preencher todos os NaNs remanescentes (do merge e de NaT) com -1
    pd.set_option('future.no_silent_downcasting', True)
    df_final = df_final.fillna(-1).infer_objects(copy=False)

    return df_final

Salva o Dataframe em um arquivo .csv

In [ ]:
def salvar_csv(dataframe: pd.DataFrame, nome_arquivo: str):

    try:
        # index=False evita que o índice do pandas seja salvo como uma coluna no csv
        # encoding='utf-8-sig' garante compatibilidade com excel para caracteres especiais
        dataframe.to_csv(nome_arquivo, index=False, encoding='utf-8-sig')
        print(f"dataframe salvo com sucesso em '{nome_arquivo}'")
    except Exception as e:
        print(f"erro ao salvar o arquivo: {e}")

Execução principal que chama as outras funções.

In [ ]:
if __name__ == '__main__':
    # definição dos arquivos de origem
    ARQUIVO_SPEEDRUN = 'speedrun_stats.csv'
    ARQUIVO_TWITCH = 'twitch_stats.csv'
    ARQUIVO_YOUTUBE = 'youtube_stats.csv'
    ARQUIVO_BILIBILI = 'bilibili_stats.csv'
    ARQUIVO_SAIDA = 'dados_analise_final.csv'

    # unificação e limpeza
    stats_unificados = unificar_estatisticas_videos(ARQUIVO_TWITCH, ARQUIVO_YOUTUBE, ARQUIVO_BILIBILI)
    dados_completos = juntar_e_limpar_dados(ARQUIVO_SPEEDRUN, stats_unificados)

    # processamento e limpeza dos dados
    stats_unificados = unificar_estatisticas_videos(ARQUIVO_TWITCH, ARQUIVO_YOUTUBE, ARQUIVO_BILIBILI)
    dados_completos = juntar_e_limpar_dados(ARQUIVO_SPEEDRUN, stats_unificados)

    # filtro para remover runs sem vídeo associado (onde 'plataforma' é -1)
    df_analise_filtrado = dados_completos[dados_completos['plataforma'] != -1].copy()

    # seleção das métricas relevantes para análise
    colunas_relevantes = [
        'run_date', 'run_player', 'run_time_seconds', 'video_link', 'plataforma',
        'video_data_publicacao', 'VideoAgeDays', 'video_views',
        'video_likes', 'video_comentarios', 'ViewsPerDay', 'EngagementRate',
        'ChannelID', 'CurrentSubscribers', 'Views_5d_Depois', 'Likes_5d_Depois',
        'danmaku', 'coins', 'shares', 'favorites'
    ]

    # filtra o dataframe completo para manter apenas colunas relevantes
    df_analise = df_analise_filtrado.reindex(columns=colunas_relevantes, fill_value=-1)

    # exibição e análise dos resultados
    print(" Dataframe de análise (apenas colunas relevantes, runs sem vídeo removidas):")
    print(df_analise.head(10))

    print("\n Informações e tipos de dados do dataframe de análise:")
    df_analise.info()

    print("\n Exemplo de runs do twitch no dataframe limpo:")
    if not df_analise[df_analise['plataforma'] == 'Twitch'].empty:
        print(df_analise[df_analise['plataforma'] == 'Twitch'].head())
    else:
        print("\n Nenhuma run do twitch encontrada após a filtragem.")


    print("\n Exemplo de runs do youtube no dataframe limpo:")
    print(df_analise[df_analise['plataforma'] == 'YouTube'].head())

    print("\n Exemplo de runs do bilibili no dataframe limpo:")
    print(df_analise[df_analise['plataforma'] == 'Bilibili'].head())

    print("\n Resumo estatístico para identificar outliers:")
    colunas_numericas = df_analise.select_dtypes(include=np.number).columns
    print(df_analise[colunas_numericas].describe().apply(lambda s: s.apply('{0:.2f}'.format)))

    print("\n--- Salvando resultado ---")
    salvar_csv(df_analise, ARQUIVO_SAIDA)




--- Processando Bilibili a partir de: bilibili_stats.csv ---
Bilibili: CSV carregado. 10 linhas iniciais.
Bilibili: 2 vídeos com 'context_video' == 'recorde'.
Bilibili: 2 recordes processados e adicionados ao DataFrame.

--- Processando Bilibili a partir de: bilibili_stats.csv ---
Bilibili: CSV carregado. 10 linhas iniciais.
Bilibili: 2 vídeos com 'context_video' == 'recorde'.
Bilibili: 2 recordes processados e adicionados ao DataFrame.
 Dataframe de análise (apenas colunas relevantes, runs sem vídeo removidas):
     run_date run_player  run_time_seconds                               video_link plataforma      video_data_publicacao  VideoAgeDays  video_views  video_likes  video_comentarios  ViewsPerDay  EngagementRate                 ChannelID  CurrentSubscribers  Views_5d_Depois  Likes_5d_Depois  danmaku  coins  shares  favorites
3  2022-06-25   1xygz7wx            3979.0  https://youtube.com/watch?v=7vycum--l3g    YouTube  2022-06-25 00:18:57+00:00        1202.0       1114.0        

/tmp/ipython-input-3019352523.py:19: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_final['video_data_publicacao'] = pd.to_datetime(df_final['video_data_publicacao'], errors='coerce')
/tmp/ipython-input-3019352523.py:19: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_final['video_data_publicacao'] = pd.to_datetime(df_final['video_data_publicacao'], errors='coerce')


COMENTÁRIO DA EQUIPE:
O scraping de speedrun.py por motivos tecnicos acabou não gerando os mesmos links utilizados no twitch_coleta_dados.py, portanto infelizmente foi necessário excluir os dados dessa plataforma.